# UTSA CS 3793/5233: Assignment-1

Fall 2021


**Murphey - Quinn - (pfl955)**






## Learning Objectives


*   Read data from a file and Create a graph
*   Implement Uninformed & Informed searching strategies
*   Apply different searching strategies for the given problem
*   Analyze and Compare the searching strategies


## Description

This assignment is focused on **python file reading, graph creation** and implementation of **search algorithms**. 
In the following sections, you will complete a series of tasks for a made up problem of *Coronavirus in Texas*.

*   Coronavirus is non-discriminatory, in the sense that it can spread from one city to any other city. The only goal of the virus is to spread to all cities in Texas. Find a possible way for the virus to spread (Uninformed Search).
*   To counter the effect of the virus, vaccine needs to be distributed to all cities. One city has more demand than supply, whereas one city has a shortage of vaccines. The goal is to find an **optimal** strategy to transport the vaccine (Informed Search) from the city with high supply (low demand) to the city with low supply (high demand).

The base structure and comments are provided on what should be done. You can use some libraries that help support you for the successful completion of the assignment. However, you **CANNOT** use a complete library for the search algorithms. You can get pieces of code from online, but please cite the source properly.


#Reading Data Files & Creating a 2D Graph

##(50 points)

In this section, you will write code to read the data files provided, cities.csv and distances.csv, and create a 2D graph consisting of nodes and edges. The goal is to use this graph for the 2 search agents that you will create in the next section.

Provided with this lab, on Blackboard, you will find 2 csv files:

*   **cities.csv** - This file contains a list of coordinates for selected cities in Texas, in the following format:
```
San Antonio,29.4685,-98.5254
```
The above line means that San Antonio is located at the latitude and longitude of 29.4685 N and 98.5254 W respectively. Note that the '-ve' sign denotes 'S' for latitude and 'W' for longitude. While performing calculations you will need to ignore the sign.

*   **distances.csv** - This file contains distance values between two cities of Texas, if a path exists, in the following format:
```
San Antonio,New Braunfels,30.80876734
```
The above line denotes that there should be an edge between *San Antonio* and *New Braunfels* and the weight on that edge, i.e. the distance, is *30.80876734*.

In the code blocks below, handle the logic for the graph. Load the graph data from the give files and display a 2D graph of the given data, with labeled nodes and edges. Create as many functions or code blocks as needed.

##Extra Credit (4 points)

Overlay the 2D graph on an image of the Texas state map.





In [1]:
# Add only your imports here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib notebook

In [2]:
# Assume that the data files are in the following folder -- THIS WILL BE USED BY THE TA
basePath = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/"
basePath = "/home/nragis/Documents/personalStudies/csclasses/3793/a1/"

In [16]:
# Load the graph data from the files

citiesRaw = pd.read_csv(basePath + "cities.csv", names=["city","lattitude","longitude"])
distancesRaw = pd.read_csv(basePath + "distances.csv", names=["cityA","cityB","distance"])

print(distancesRaw)

              cityA            cityB    distance
0        San Angelo          Midland  112.283423
1        San Angelo          Lubbock  185.113579
2        San Angelo          Abilene   95.269070
3        San Angelo      San Antonio  210.849482
4       San Antonio    New Braunfels   30.808767
5       San Antonio           Seguin   33.992046
6       San Antonio     Three Rivers   74.509098
7       San Antonio           Uvalde   82.709939
8            Austin       San Marcos   30.717426
9            Austin       Round Rock   18.464010
10           Austin  College Station  106.758822
11           Austin          Houston  165.907654
12           Temple             Waco   35.942526
13  College Station             Waco   97.827871
14  College Station          Houston   99.209177
15          Houston         Beaumont   99.579513
16          Houston        Galveston   65.021400
17          Houston         Columbus   72.515122
18          Houston       Sugar Land   59.135783
19         Victoria 

In [4]:
# Display a 2D graph of the given data.

x = citiesRaw['longitude']
y = citiesRaw['lattitude']
n = citiesRaw['city']

# Print image of TX
img = plt.imread("texas.jpg")
plt.imshow(img, zorder=0, extent=[-106.7, -93.4, 25.837377, 36.500704], aspect='auto')
# bounding box of texas from https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/

# scatter plot cities and labels
plt.scatter(x,y,zorder=1)
for i, txt in enumerate(citiesRaw['city']):
    plt.annotate(txt, (x[i]+.15, y[i]-0.1), fontsize=6)

# plot all lines
for _, row in distancesRaw.iterrows():
    cityAIndex = n[n  == row['cityA']].index.tolist()[0]
    Ax = x[cityAIndex]
    Ay = y[cityAIndex]
    cityBIndex = n[n  == row['cityB']].index.tolist()[0]
    Bx = x[cityBIndex]
    By = y[cityBIndex]
    plt.plot([Ax,Bx],[Ay,By])
    plt.annotate(round(row['distance'],1), ( (Ax + Bx)/2 , (Ay + By)/2 ), fontsize=6)


plt.xlabel("Longitude")
plt.xlabel("Lattitude")
plt.title("Texas Cities & Distances")
plt.show()

#drag bottom right corner to increase size for readability

<IPython.core.display.Javascript object>

#Virus Spread - Uninformed Search Agent

##(50 points)

In this section, you will use the graph created in the previous section and create an *uninformed search* agent that will print the path how the virus will spread to all the provided Texas cities. The first confirmed case of the virus was in **Three Rivers** and starts spreading from there. The virus does not discriminate and it needs to spread to all the cities of Texas.

In the following code block, write code to implement **any** uninformed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path or sequence of cities that will be infected by the spread of Coronavirus.
*   The distance travelled by the selected virus spreading strategy.

##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [5]:
# Implement ANY uninformed search strategy for the spread of coronavirus from the starting city of 'Three Rivers'

visitedNodes = []

def dfs(nodes, edges, start, end, prevNodes=[]):
    """
    Params:
        nodes: dataFrame with two columns (longitude, lattitude) and a row index (name)
        edges: dataFrame with 3 columns (cityA, cityB, distance) and a numeric index
        start: name of a node
        end: name of a node
    
    Returns:
        path: A list of all cities travelled including the start and end
        dist: The total distance traveled
    """
    if(start not in visitedNodes):
        visitedNodes.append(start)
    
    if start == end:
        return ([end], 0)
    
    searchEdges = edges.loc[ (edges['cityA'] == start) | (edges['cityB'] == start) ]
    for _, row in searchEdges.iterrows():
        if row['cityA'] == start:
            nextNode = row['cityB']
        else:
            nextNode = row['cityA']
        
        if nextNode in prevNodes:
            continue
        dist = row['distance']
        
        newPrevNodes = prevNodes.copy()
        newPrevNodes.append(start)
        if output := dfs(nodes, edges, nextNode, end, newPrevNodes):
            p = output[0]
            d = output[1] + dist
            if p[-1] == end:
                p.insert(0, start)
                return p,d
        

# Prepare data
nodes = citiesRaw.set_index('city')
edges = distancesRaw

# Run dfs
for name in nodes.index.tolist():
    path, dist = dfs(nodes, edges, "Three Rivers", name)
    
    print(name + ":")
    
    print("\tPath: ", end='')
    for i in range(len(path)-1):
        print(path[i] + " -> ", end='')
    print(path[-1])
    
    print("\tDistance: " + str(dist))

print()
print("****************************")
print()
print("Order of infection (by dfs):")
for city in visitedNodes:
    print("\t" + city)

Abilene:
	Path: Three Rivers -> San Antonio -> San Angelo -> Abilene
	Distance: 380.62764993999997
Alice:
	Path: Three Rivers -> Alice
	Distance: 51.26861733
Amarillo:
	Path: Three Rivers -> San Antonio -> San Angelo -> Midland -> Lubbock -> Amarillo
	Distance: 637.71513777
Austin:
	Path: Three Rivers -> San Antonio -> New Braunfels -> San Marcos -> Austin
	Distance: 154.52477707999998
Beaumont:
	Path: Three Rivers -> San Antonio -> New Braunfels -> San Marcos -> Austin -> Round Rock -> Temple -> Waco -> College Station -> Houston -> Beaumont
	Distance: 555.2717662499999
Brownsville:
	Path: Three Rivers -> Alice -> Laredo -> McAllen -> Brownsville
	Distance: 353.31905978
College Station:
	Path: Three Rivers -> San Antonio -> New Braunfels -> San Marcos -> Austin -> Round Rock -> Temple -> Waco -> College Station
	Distance: 356.48307585
Columbus:
	Path: Three Rivers -> San Antonio -> New Braunfels -> San Marcos -> Austin -> Round Rock -> Temple -> Waco -> College Station -> Houston -> C

#Vaccine Transportation - Informed Search Agent

##(50 points)

In this section, you will create an *informed search* agent that will be used to transport the vaccine. The city of **San Antonio** has more supply of vaccine than the demand. The goal is to create an **optimal strategy** to transport the vaccine and make it available at the highly affected city of **College Station**, where there is a shortage of vaccines.

In the following code block, write code to implement an **optimal** informed search strategy. You are free to create more code blocks if needed. As the output, print 

*   The path / sequence of cities that will be visited in the optimal vaccine transportation strategy.
*   The total distance travelled in the optimal vaccine transportation strategy.


##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [17]:
# Implement an OPTIMAL informed search strategy for distributing the vaccine from 'San Antonio' to 'College Station'

def a_star(nodes, edges, start, end):
    def h(node):
        startRow = nodes.loc[start]
        x1 = startRow['longitude']
        y1 = startRow['lattitude']
        endRow = nodes.loc[node]
        x2 = endRow['longitude']
        y2 = endRow['lattitude']
        
        return math.sqrt( (x1 - x2)**2 + (y1 - y2)**2 )
    
    def d(start,end):
        edge = edges.loc[ ((edges['cityA'] == start) & ((edges['cityB'] == end))) | 
                    ((edges['cityA'] == end) & ((edges['cityB'] == start))) ]
        if len(edge) == 1:
            return edge.iloc[0]['distance']
        else:
            return -1
    
    def path_dist(path):
        if len(path) <= 1:
            return 0
        
        s = 0
        for i in range(len(path) - 1):
            s += d(path[i],path[i+1])
        return s
    
    def retrace_path(node):
        currNode = node
        path = [node]
        while currNode != start:
            nextNode = cameFrom[currNode]
            path.insert(0, nextNode)
            currNode = nextNode
        return path
    
    def get_neighbors(node):
        neighborsEdges = edges.loc[ (edges['cityA'] == node) | (edges['cityB'] == node) ]
        neighbors = []
        for _, row in neighborsEdges.iterrows():
            if row['cityA'] == node:
                neighbors.append(row['cityB'])
            else:
                neighbors.append(row['cityA'])
                
        return neighbors
    """
    Params:
        nodes: dataFrame with two columns (longitude, lattitude) and a row index (name)
        edges: dataFrame with 3 columns (cityA, cityB, distance) and a numeric index
        start: name of a node
        end: name of a node
    
    Returns:
        path: A list of all cities travelled including the start and end
        dist: The total distance traveled
    """
    openSet = [start]
    cameFrom = {}
    gScore = {}
    gScore[start] = 0
    fScore = {}
    fScore[start] = h(start)
    current = start
    while openSet:
        current = openSet[0]
        minF = fScore.get(openSet[0], math.inf)
        for node in openSet:
            if fScore.get(node, math.inf) < minF:
                currentNode = node
                minF = fScore.get(node, math.inf)
        
        #print(current)
        if current == end:
            path = retrace_path(current)
            return path, path_dist(path)

        openSet.remove(current)
        
        for neighbor in get_neighbors(current):
            tentative_gScore = gScore.get(current, math.inf) + d(current, neighbor)
            if tentative_gScore < gScore.get(neighbor, math.inf):
                cameFrom[neighbor] = current
                gScore[neighbor] = tentative_gScore
                fScore[neighbor] = gScore.get(neighbor, math.inf) + h(neighbor)
                if neighbor not in openSet:
                    openSet.append(neighbor)

    return None, None

nodes = citiesRaw.set_index('city')
edges = distancesRaw

path, dist = a_star(nodes, edges, "San Antonio", "College Station")

print("Path:")
for city in path:
    print("\t" + city)
print("Distance: \n\t" + str(dist))

Path:
	San Antonio
	New Braunfels
	San Marcos
	Austin
	College Station
Distance: 
	186.77450141


#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard

##Rubric

*   (50 points) Reading Data files & Creating a 2D Graph
*   (50 points) Virus Spread - Uninformed Search Agent
*   (50 points) Vaccine Transportation - Informed Search Agent
*   (10 points) Extra Credit - on the Texas map image, overlay the 2D graph and the paths selected by the search agents



